#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [19]:
import pandas as pd
import pymongo
import json

In [22]:
client = pymongo.MongoClient('127.0.0.1', 27017) # Si utilisation en local
#client = pymongo.MongoClient('mongo') # Si utilisation d'une image Docker
database = client['exercices']
collection_ks = database['kickstarter']

In [23]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


In [24]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [25]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                150000 non-null  int64  
 1   name              149998 non-null  object 
 2   category          150000 non-null  object 
 3   main_category     150000 non-null  object 
 4   currency          150000 non-null  object 
 5   deadline          150000 non-null  object 
 6   goal              150000 non-null  object 
 7   launched          150000 non-null  object 
 8   pledged           150000 non-null  object 
 9   state             150000 non-null  object 
 10  backers           150000 non-null  object 
 11  country           150000 non-null  object 
 12  usd pledged       148518 non-null  object 
 13  usd_pledged_real  150000 non-null  float64
dtypes: float64(1), int64(1), object(12)
memory usage: 16.0+ MB


## Question 0

### Netoyer les données

In [26]:
df_ks = df_ks.rename(columns = {'usd pledged':'usd_pledged'})

In [27]:
lst_col_trait = ['goal', 'pledged', 'backers', 'usd_pledged']
col_name = 'name'

In [28]:
def traitement_table(df, lst_col_trait, name):
    for i in lst_col_trait:
        row_to_supp_goal = df[pd.to_numeric(df[i], errors ='coerce').isnull()].index
        df = df.drop(index = row_to_supp_goal)
        df[i] = df[i].astype(float)
    df[name] = df[name].astype('str')
    return df

In [29]:
df_ks = traitement_table(df_ks, lst_col_trait, col_name)

In [30]:
df_ks = df_ks.dropna()

In [31]:
df_ks.drop_duplicates(subset = 'name', keep = 'first', inplace=True)

In [32]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148015 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                148015 non-null  int64  
 1   name              148015 non-null  object 
 2   category          148015 non-null  object 
 3   main_category     148015 non-null  object 
 4   currency          148015 non-null  object 
 5   deadline          148015 non-null  object 
 6   goal              148015 non-null  float64
 7   launched          148015 non-null  object 
 8   pledged           148015 non-null  float64
 9   state             148015 non-null  object 
 10  backers           148015 non-null  float64
 11  country           148015 non-null  object 
 12  usd_pledged       148015 non-null  float64
 13  usd_pledged_real  148015 non-null  float64
dtypes: float64(5), int64(1), object(8)
memory usage: 16.9+ MB


In [33]:
df_ks

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.00,canceled,24.0,US,1145.000000,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.00,failed,28.0,US,1851.000000,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.00,successful,254.0,US,3796.000000,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.00,successful,367.0,US,2670.000000,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.00,failed,1.0,CA,0.708148,0.738225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,574841011,Scutes Headphones - A new way to listen to you...,Product Design,Design,USD,2014-11-14,40000.0,2014-10-14 20:44:37,3664.00,canceled,43.0,US,3664.000000,3664.000000
149996,235190237,My Drunk @$$ Reviews,Webseries,Film & Video,USD,2012-05-18,1000.0,2012-04-18 04:42:47,0.00,failed,0.0,US,0.000000,0.000000
149997,758540498,Freedom Planet - High Speed Platform Game,Video Games,Games,USD,2013-02-15,2000.0,2013-01-16 00:53:32,25472.69,successful,1105.0,US,25472.690000,25472.690000
149998,1624952469,Clown vs Mime,Webseries,Film & Video,USD,2011-05-27,1000.0,2011-04-27 01:22:52,76.00,failed,3.0,US,76.000000,76.000000


In [34]:
dtypeCount = [df_ks.iloc[:,i].apply(type).value_counts() for i in range(df_ks.shape[1])]

In [35]:
dtypeCount

[<class 'int'>    148015
 Name: ID, dtype: int64, <class 'str'>    148015
 Name: name, dtype: int64, <class 'str'>    148015
 Name: category, dtype: int64, <class 'str'>    148015
 Name: main_category, dtype: int64, <class 'str'>    148015
 Name: currency, dtype: int64, <class 'str'>    148015
 Name: deadline, dtype: int64, <class 'float'>    148015
 Name: goal, dtype: int64, <class 'str'>    148015
 Name: launched, dtype: int64, <class 'float'>    148015
 Name: pledged, dtype: int64, <class 'str'>    148015
 Name: state, dtype: int64, <class 'float'>    148015
 Name: backers, dtype: int64, <class 'str'>    148015
 Name: country, dtype: int64, <class 'float'>    148015
 Name: usd_pledged, dtype: int64, <class 'float'>    148015
 Name: usd_pledged_real, dtype: int64]

### Importer les données

In [36]:
df_ks_dict = df_ks.to_dict('records')

In [37]:
df_ks_dict

[{'ID': 872782264,
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
  'category': 'Rock',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2011-09-16',
  'goal': 2000.0,
  'launched': '2011-08-17 06:31:31',
  'pledged': 1145.0,
  'state': 'canceled',
  'backers': 24.0,
  'country': 'US',
  'usd_pledged': 1145.0,
  'usd_pledged_real': 1145.0},
 {'ID': 1326492673,
  'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'currency': 'USD',
  'deadline': '2012-08-22',
  'goal': 18000.0,
  'launched': '2012-07-23 20:46:48',
  'pledged': 1851.0,
  'state': 'failed',
  'backers': 28.0,
  'country': 'US',
  'usd_pledged': 1851.0,
  'usd_pledged_real': 1851.0},
 {'ID': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2016-07-19',
  'goal': 2000.0,
  'launched': '2016-07-01 21:55:54',
  'pledged': 7534.0,
  '

In [38]:
collection_ks.drop()

In [39]:
collection_ks.insert_many(df_ks_dict)

In [40]:
print(client.list_database_names())

['admin', 'config', 'exercices', 'local']


## Question 1  

In [41]:
cur_ex1 = collection_ks.find().sort([("pledged", -1)]).limit(5)

In [42]:
list(cur_ex1)

[{'_id': ObjectId('601d0a42941a132c4d2557cd'),
  'ID': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30',
  'goal': 50000.0,
  'launched': '2014-07-08 10:14:37',
  'pledged': 13285226.36,
  'state': 'successful',
  'backers': 62642.0,
  'country': 'US',
  'usd_pledged': 13285226.36,
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('601d0a42941a132c4d25b19c'),
  'ID': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30',
  'goal': 1000000.0,
  'launched': '2016-05-24 15:49:52',
  'pledged': 12779843.49,
  'state': 'successful',
  'backers': 66673.0,
  'country': 'US',
  'usd_pledged': 12779843.49,
  'usd_pledged_real': 12779843.49},
 {'_id': ObjectId('601d0a42941a132c4d2563da'),
  'ID': 2111201788,
  'name': 'Expect the Unex

## Question 2

In [43]:
collection_ks.count_documents({"state" : "successful"})

52832

## Question 3

In [44]:
cur_ex3 = collection_ks.aggregate([{'$group' : {'_id': '$category', 'countByCategory' : {'$sum' : 1}}}])

In [45]:
list(cur_ex3)

[{'_id': 'Civic Design', 'countByCategory': 130},
 {'_id': 'Weaving', 'countByCategory': 38},
 {'_id': 'Games', 'countByCategory': 1322},
 {'_id': 'Embroidery', 'countByCategory': 49},
 {'_id': 'Metal', 'countByCategory': 273},
 {'_id': 'Latin', 'countByCategory': 61},
 {'_id': 'Video', 'countByCategory': 145},
 {'_id': 'Community Gardens', 'countByCategory': 114},
 {'_id': 'Conceptual Art', 'countByCategory': 392},
 {'_id': 'Wearables', 'countByCategory': 508},
 {'_id': 'Indie Rock', 'countByCategory': 2187},
 {'_id': 'Drama', 'countByCategory': 861},
 {'_id': 'Nonfiction', 'countByCategory': 3385},
 {'_id': 'Publishing', 'countByCategory': 2141},
 {'_id': 'Rock', 'countByCategory': 2700},
 {'_id': 'Country & Folk', 'countByCategory': 1787},
 {'_id': 'Science Fiction', 'countByCategory': 269},
 {'_id': 'Accessories', 'countByCategory': 1200},
 {'_id': 'Textiles', 'countByCategory': 105},
 {'_id': 'Playing Cards', 'countByCategory': 960},
 {'_id': 'Tabletop Games', 'countByCategory': 5

In [46]:
cur_ex3 = collection_ks.aggregate([{'$group' : {'_id': '$main_category', 'count_By_main_Category' : {'$sum' : 1}}}])

In [47]:
list(cur_ex3)

[{'_id': 'Film & Video', 'count_By_main_Category': 24821},
 {'_id': 'Dance', 'count_By_main_Category': 1486},
 {'_id': 'Photography', 'count_By_main_Category': 4165},
 {'_id': 'Theater', 'count_By_main_Category': 4285},
 {'_id': 'Journalism', 'count_By_main_Category': 1803},
 {'_id': 'Art', 'count_By_main_Category': 11228},
 {'_id': 'Fashion', 'count_By_main_Category': 8934},
 {'_id': 'Publishing', 'count_By_main_Category': 15679},
 {'_id': 'Music', 'count_By_main_Category': 19596},
 {'_id': 'Design', 'count_By_main_Category': 11888},
 {'_id': 'Food', 'count_By_main_Category': 9775},
 {'_id': 'Technology', 'count_By_main_Category': 12850},
 {'_id': 'Crafts', 'count_By_main_Category': 3453},
 {'_id': 'Games', 'count_By_main_Category': 13922},
 {'_id': 'Comics', 'count_By_main_Category': 4130}]

## Question 4

In [48]:
collection_ks.count_documents({"$and" : [{"country":'FR'}, {"launched":{'$lt' : '2016-01-01'}}]})

329

## Question 5

In [49]:
cur_ex5 = collection_ks.find({"$and" : [{"country":'US'}, {"usd_pledged":{'$gte' : 200000}}]})

In [50]:
list(cur_ex5)

[{'_id': ObjectId('601d0a42941a132c4d24306a'),
  'ID': 217543389,
  'name': 'The uKeg Pressurized Growler for Fresh Beer',
  'category': 'Drinks',
  'main_category': 'Food',
  'currency': 'USD',
  'deadline': '2014-12-08',
  'goal': 75000.0,
  'launched': '2014-10-15 06:34:48',
  'pledged': 1559525.68,
  'state': 'successful',
  'backers': 10293.0,
  'country': 'US',
  'usd_pledged': 1559525.68,
  'usd_pledged_real': 1559525.68},
 {'_id': ObjectId('601d0a42941a132c4d2430f9'),
  'ID': 909248984,
  'name': 'Redux COURG - Hybrid Watches with Missions to Tackle',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2015-08-20',
  'goal': 30000.0,
  'launched': '2015-07-21 19:01:41',
  'pledged': 692912.0,
  'state': 'successful',
  'backers': 2200.0,
  'country': 'US',
  'usd_pledged': 692912.0,
  'usd_pledged_real': 692912.0},
 {'_id': ObjectId('601d0a42941a132c4d2431a6'),
  'ID': 1688905333,
  'name': 'Legion Solar - A Better Way to Energy Inde

In [51]:
collection_ks.count_documents({"$and" : [{"country":'US'}, {"usd_pledged":{'$gte' : 200000}}]})

461

## Question 6 

In [52]:
collection_ks.create_index([("name",  "text")])

'name_text'

In [53]:
collection_ks.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [54]:
collection_ks.count_documents( { "$text": { "$search": "Sport" } } )

316